![ine-divider](https://user-images.githubusercontent.com/7065401/92672068-398e8080-f2ee-11ea-82d6-ad53f7feb5c0.png)
<hr>

### HTTP using Python

# Creating Web Servers

This project will have you create a web servers with some special functionality, built incrementally.  For the project, you will use the Flask framework, which is more modern and flexible than the standard libary module.

![orange-divider](https://user-images.githubusercontent.com/7065401/92672455-187a5f80-f2ef-11ea-890c-40be9474f7b7.png)

## Part 1

**Browse file paths**

The `http.server` libary had a zero-configuration mode where it could simply act as a file browser of a local directory, allowing that to be served remotely as well.  We would like to implement something analogous in Flask.  Each file in the local directory should be shown as a name and a link, and viewers should return files with an appropriate file type.

In evaluating your server, you will probably wish to point a regular web browser to it, and navigate dynamically, rather than only making code requests against the server.  Since this server will accept paths matching files that may be dynamically created, you should read about [URL Route Registrations](https://flask.palletsprojects.com/en/2.0.x/api/#url-route-registrations) to understand how this is possible.

For illustration, however, here are some possible behaviors, shown in this notebook.

```python
>>> resp = requests.get('http://localhost:2553/bad/filepath')
>>> resp.text
'bad/filepath is not a file path!'
```

```python
>>> resp = requests.get('http://localhost:2553/data/3001.json')
>>> for k, v in resp.headers.items():
...     print(f"{k}: {v}")
Content-Disposition: inline; filename=3001.json
Content-Type: application/json
Content-Length: 285
Last-Modified: Wed, 16 Jun 2021 17:36:55 GMT
Date: Wed, 16 Jun 2021 19:33:16 GMT

>>> json.loads(resp.text)
{'1': {'name': 'Guido van Rossum',
  'password': 'unladenswallow',
  'details': {'profession': 'ex-BDFL'}},
 '2': {'name': 'Brendan Eich',
  'password': 'nontransitiveequality',
  'details': {'profession': 'Mozillan'}},
 '3': {'name': 'Ken Thompson',
  'password': 'p/q2-q4!',
  'details': {'profession': 'Unix Creator'}}}
```

```python
>>> from IPython.core.display import HTML
>>> resp = requests.get('http://localhost:2553/images/')
>>> HTML(resp.text)
```
<h2>Tree from directory images/</h2>
<pre>
├── <a href="images/rainbow-butterfly-unicorn-kitten.jpg">rainbow-butterfly-unicorn-kitten.jpg</a>
└── <a href="images/cannot-brain.jpg">cannot-brain.jpg</a>
</pre>

```python
>>> resp = requests.get('http://localhost:2553/images/cannot-brain.jpg')
>>> for k, v in resp.headers.items():
...     print(f"{k}: {v}")
Content-Disposition: inline; filename=cannot-brain.jpg
Content-Type: image/jpeg
Content-Length: 23058
Last-Modified: Wed, 16 Jun 2021 17:40:19 GMT
Cache-Control: no-cache
Date: Wed, 16 Jun 2021 19:36:08 GMT
```

---
## Solution

The particular appearance of pages returned by your file browsing web server can vary depending on your needs.  For example, an elaborate version might include icons varying by file type, or data about files such as size or creation date.  However, the following is a functional implementation showing much of what you will want in your version.

```python
from pathlib import Path
from urllib.parse import quote
from mimetypes import guess_type

from flask import Flask, request, Response, send_file
app = Flask(__name__)

# prefix components:
space =  '    '
branch = '│   '
tee =    '├── '
last =   '└── '

def linkify(pth):
    return f'<a href="{quote(str(pth))}">{pth.name}</a>'

def make_tree(path, prefix='', linkify=lambda pth: pth):
    path = Path(path)
    contents = list(path.iterdir())
    pointers = [tee] * (len(contents) - 1) + [last]
    for pointer, path in zip(pointers, contents):
        if path.name.startswith(('.', '__pycache__')):
            continue
        yield (f"{prefix}{pointer}" +
               f"{linkify(path)}" +
               f"{'/' if path.is_dir() else ''}")
        if path.is_dir(): # extend the prefix and recurse:
            extension = branch if pointer == tee else space 
            yield from make_tree(path, prefix=prefix+extension, linkify=linkify)

dirpage = """<html>
<head><title>Directory of %s/</title></head>
<body>
<h2>Tree from directory %s/</h2>
<pre>
%s
</pre></body>
</html>
"""

@app.route('/')
def basepath():
    root = "."
    tree = '\n'.join(make_tree('.', linkify=linkify))
    return Response(dirpage % (root, root, tree), mimetype='text/html')

@app.route('/<path:filepath>')
def get_path(filepath):
    q = Path(filepath)
    if q.exists():
        if q.is_dir():
            root = q.name
            tree = '\n'.join(make_tree(root, linkify=linkify))
            return Response(dirpage % (root, root, tree))
        else:
            filetype = guess_type(filepath)[0]
            return send_file(filepath, mimetype=filetype)
    else:
        return f"{filepath} is not a file path!"
```

![orange-divider](https://user-images.githubusercontent.com/7065401/92672455-187a5f80-f2ef-11ea-890c-40be9474f7b7.png)

## Part 2

**Display file metadata**

Under Unix-like systems, we might find the size of all the files within a directory hierarchy with a command such as this:

```bash
$ find . -type f | xargs du -b
9405    ./psf-logo.png
1969    ./server-solution.py
214     ./hello.html
375     ./greetings.txt
285     ./data/3001.json
5469    ./data/olympic-medals.csv
604     ./data/example.yaml
24706   ./images/rainbow-butterfly-unicorn-kitten.jpg
23058   ./images/cannot-brain.jpg
```

For this task, you should add an additional route within the **same** server your wrote in the first part, which will return a JSON representation of this information.  Use the route `/file-sizes` to retrieve this information; note that if a file called `file-sizes` existed, this would create an ambiguigy, but for this purpose simply assume that such file does not exist.

A working version might behave like:

```python
>>> resp = requests.get('http://localhost:2553/file-sizes')
>>> json.loads(resp.text)
{'./data/3001.json': 285,
 './data/example.yaml': 604,
 './data/olympic-medals.csv': 5469,
 './greetings.txt': 375,
 './hello.html': 214,
 './images/cannot-brain.jpg': 23058,
 './images/rainbow-butterfly-unicorn-kitten.jpg': 24706,
 './psf-logo.png': 9405,
 './server-solution.py': 2340}
```

---
## Solution

The following route added to the initial server can fulfill this task.  This excludes a couple paths that we do not want to report on that are generated by working in notebooks in this directory.

```python
import os
from os.path import join, getsize
    
@app.route('/file-sizes')
def file_sizes():
    sizes = {}
    for root, dirs, files in os.walk('.'):
        if '.ipynb' not in root and '__pycache__' not in root:
            for fname in files:
                fpath = join(root, fname)
                sizes[fpath] = getsize(fpath)
    return jsonify(sizes)
```

![orange-divider](https://user-images.githubusercontent.com/7065401/92672455-187a5f80-f2ef-11ea-890c-40be9474f7b7.png)